# Imputation time series

> Impute time series using GPFA Leaner

the goal of this notebook is to be able to:

- take a timeseries that contains gaps
- train a GPFA Learner using the available data
- impute the gap, by using the predictions of the learner
- to improve the imputation by conditioning the distribution on the other observation at the same time stamp

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| hide
#| default_exp imputation

In [3]:
from gpfa_imputation.learner import *

import torch

## Fake data 

generate some fake data in order to test the imputation

What is does is:
- take a function to generate the "true" latent
- use some random coefficient to generate all the N features
- add some random noise

In [37]:
#| export
class GPFAFakeData:
    def __init__(self,
                    n_features: int,
                    n_obs: int,
                    latent_func = torch.sin, # Functions used to generate the true latent:
                    noise_std = .2
                ):
        
        self.n_features, self.n_obs = n_features, n_obs
        self.T = torch.arange(n_obs)
        
        self.latent = latent_func(self.T)
        
        self.Lambda = torch.rand(n_features, 1)
        
        self.exact_X = (self.Lambda * self.latent).T
        
        self.X =  self.exact_X + torch.normal(0., noise_std, size = (n_obs, n_features)) 
        

In [38]:
fd = GPFAFakeData(3, 10)

In [39]:
fd.X

tensor([[ 0.1093, -0.0152,  0.2280],
        [ 0.3716,  0.3290,  1.1037],
        [ 0.3686, -0.0628,  0.8996],
        [-0.0736, -0.1601, -0.0065],
        [-0.4442,  0.1232, -0.6697],
        [ 0.0191, -0.1219, -0.6036],
        [ 0.1061,  0.1039, -0.3509],
        [ 0.2677, -0.1632,  0.7693],
        [ 0.5997,  0.1350,  0.8118],
        [-0.1333,  0.0640,  0.1867]])

In [40]:
fd.exact_X

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.2660,  0.1493,  0.7628],
        [ 0.2875,  0.1613,  0.8243],
        [ 0.0446,  0.0250,  0.1279],
        [-0.2393, -0.1342, -0.6860],
        [-0.3032, -0.1701, -0.8692],
        [-0.0883, -0.0496, -0.2533],
        [ 0.2077,  0.1165,  0.5955],
        [ 0.3128,  0.1755,  0.8968],
        [ 0.1303,  0.0731,  0.3736]])

In [41]:
#| hide
from nbdev import nbdev_export
nbdev_export()